**Импорт данных**

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def parse_json_line(json_line, id_counter):
    json_data = json.loads(json_line)
    text = json_data['text']
    comments = json_data['comments']
    comments_text = [comment['text'] for comment in comments]
    comments_score = [comment['score'] for comment in comments]
    id_value = next(id_counter)
    return pd.DataFrame({'id': [id_value]*5, 'text': [text]*5, 'comment': comments_text, 'score': comments_score})

with open('/kaggle/input/vk-ds-cup/ranking_train.jsonl', 'r') as f:
    train_data = list(f)

with open('/kaggle/input/vk-ds-cup/ranking_test.jsonl', 'r') as f:
    test_data = list(f)

id_counter = iter(range(1, len(train_data)+len(test_data)+1))

train_df_list = []
for line in train_data:
    train_df_list.append(parse_json_line(line, id_counter))

train_df = pd.concat(train_df_list, ignore_index=True)

test_df_list = []
for line in test_data:
    test_df_list.append(parse_json_line(line, id_counter))

test_df = pd.concat(test_df_list, ignore_index=True)

In [ ]:
# Если уже есть данные
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

train_df = pd.read_csv('/kaggle/working/test_for_model.csv')
# train_df = test_df # чтобы не менять все дальше

In [ ]:
# installation enchant in linux
# %%capture installation_linux.log # wtf перестала работать в kaggle

!sudo apt-get -y update > update.log
!sudo apt-get -y install enchant > enchant.log
!sudo apt-get -y install enchant-2 >> enchant.log
!sudo apt-get install -y hunspell-en-us >> enchant.log



In [ ]:
# %%capture installation.log
import os
def install_java():
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
  !java -version
%env LC_ALL=C.UTF-8
%env LANG=C.UTF-8
install_java()
!pip install spacy > installation.log 2>&1
!pip install language_tool_python >> installation.log 2>&1
!python -m spacy download en_core_web_sm >> installation.log 2>&1
!pip install transformers >> installation.log 2>&1
!pip install tqdm >> installation.log 2>&1
!pip install psutil >> installation.log 2>&1
!pip install Cython >> installation.log 2>&1
!pip install tensorflow-hub >> installation.log 2>&1
!pip install memory_profiler >> installation.log 2>&1
!pip install pyenchant >> installation.log 2>&1
!pip install inflect >> installation.log 2>&1
!pip install spellchecker >> installation.log 2>&1
!pip install demoji >> installation.log 2>&1
!pip install jax jaxlib >> installation.log 2>&1
!pip install qgrid >> installation.log 2>&1

In [ ]:
import psutil
# Возвращает количество ядер CPU
max_workers = psutil.cpu_count(logical=True)
print(max_workers)

In [ ]:
# Импорт библиотек
# %%capture analyze_comments.log
import re
import csv
import torch
import string
import html
import numpy as np
import spacy
import tensorflow as tf
import tensorflow_hub as hub
import nltk
import multiprocessing as mp
import enchant
import language_tool_python
import concurrent.futures as futures
import cProfile
import concurrent.futures
import demoji
import inflect
import multiprocessing
import dask.dataframe as dd
import qgrid
from cython import compile
from functools import partial
from dask.multiprocessing import get
from nltk.tokenize import word_tokenize
from memory_profiler import profile
from string import punctuation
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from textblob import TextBlob
from dask.diagnostics import ProgressBar
from numba import jit
from concurrent.futures import ProcessPoolExecutor, as_completed
from functools import partial
from tqdm import tqdm
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from transformers import AutoModelForSequenceClassification, AutoModel, AutoTokenizer, TextClassificationPipeline
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import rankdata

In [ ]:
# Загрузка pre-trained моделей
stop_words = set(stopwords.words('english'))
dict_ench = enchant.Dict("en_US")
tool = language_tool_python.LanguageTool('en-US')

# nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
# use = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
# sia = SentimentIntensityAnalyzer()
# # Инициация токенайзера
# model_path = "martin-ha/toxic-comment-model"
# tokenizer = AutoTokenizer.from_pretrained(model_path)
# model = AutoModelForSequenceClassification.from_pretrained(model_path)
# pipeline = TextClassificationPipeline(model=model, tokenizer=tokenizer)
# tokenizer_d = AutoTokenizer.from_pretrained('distilbert-base-multilingual-cased')
# model_d = AutoModel.from_pretrained('distilbert-base-multilingual-cased')

In [ ]:
# Добыча стандартных фич
def extract_links_and_clean_text(text):
    if not text:
        return '', '', 0
    pattern = r'\b(?:https?://|www\.|[a-z0-9.\-]+\.[a-z]{2,4}/)(?:[^\s()<>]+|\([^\s()<>]+\))+(?:\([^\s()<>]+\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’])'
    links = re.findall(pattern, text)
    if not links:
        return text, '', 0
    placeholder = 'LINK'
    for link in links:
        cleaned_text = text.replace(link, placeholder)
    links_string = ' '.join(links)
    num_links = len(links)
    return cleaned_text, links_string, num_links

def analyze_comment(comment, post_text, score, id):
    
    num_chars_com = len(comment)
    num_chars_post = len(post_text)
    
#     num_links = extract_links_and_clean_text(comment)

    # Find total number of spaces
    num_spaces = len(re.findall(r"\s", comment))

    # Find number of digits
    num_digits = sum([1 for char in comment if char.isdigit()])
    
    # Find number of words
    words = comment.split()
    num_words = len(words)
    
    # Find number of unique words
    unique_words = set(words)
    num_unique_words = len(unique_words)
    
    # Find number of letters and punctuation marks
    num_letters = sum([len(word) for word in words])
    num_punctuation = sum([1 for char in comment if char in string.punctuation])
    
    # Find number of uppercase letters and words
    num_uppercase_letters = sum([1 for char in comment if char.isupper()])
    num_uppercase_words = sum([1 for word in words if word.isupper()])
    
    # Find number of stop words
    num_stop_words = sum([1 for word in comment.lower().split() if word in stop_words])
    
    # Find average word length
    if num_words > 0:
        avg_word_length = num_letters / num_words
    else:
        avg_word_length = 0
    
    # Find number of sentences
    END_OF_SENTENCE_REGEX = r"[.!?;\\n]"
    sentences = re.split(END_OF_SENTENCE_REGEX, comment)
    sentences = list(filter(lambda x: x.strip(), sentences))
    num_sentences = len(sentences)

    # Find number of emojis
    num_emoji = len(demoji.findall(comment).keys())
    
    return (id,post_text,comment,score,num_chars_com,num_chars_post,num_spaces,
            num_digits,num_words,num_unique_words,avg_word_length,num_punctuation,
            num_uppercase_letters,num_uppercase_words,num_stop_words,num_sentences,
            comment.count("#"),sum(1 for word in comment.split() if word.isupper() and len(word) >= 2),
            comment.lower().count('Edit'),sum([1 for word in comment.split() if not word.isascii()]), 
            num_punctuation / len(comment),num_emoji)

def analyze_comments(df, max_workers, chunk_size=1000):

    results = np.empty((len(df), 22), dtype=object)
    
    # Распараллеливание процессов
    with futures.ProcessPoolExecutor(max_workers=max_workers) as executor:
        futures_list = []
        i = 0
        for chunk_df in tqdm(np.array_split(df, len(df) // chunk_size + 1), total=len(df) // chunk_size + 1):
            for row in chunk_df.itertuples():
                comment = row.comment
                post_text = row.text
                score = row.score
                id = row.id
                future = executor.submit(analyze_comment, comment, post_text, score, id)
                futures_list.append(future)
                i += 1
                if i % chunk_size == 0:
                    # Ожидание, пока не завершится 
                    results_list = []
                    for future in futures.as_completed(futures_list):
                        try:
                            result = future.result()
                            results_list.append(result)
                        except Exception as e:
                            print(f"Error occurred while processing task: {e}")
                    results[i-chunk_size:i] = np.vstack(results_list)
                    futures_list.clear()
        print("First loop successfully.")
        if futures_list:
            results_list = []
            for future in tqdm(futures.as_completed(futures_list), total=len(futures_list)):
                try:
                    result = future.result()
                    results_list.append(result)
                except Exception as e:
                    print(f"Error occurred while processing task: {e}")
            results[i-len(futures_list):i] = np.vstack(results_list)

    print("Analysis completed successfully.") 

    result_df = pd.DataFrame(results, columns=['id', 'post_text', 'comment', 'score', 'num_chars_com', 'num_chars_post', 'num_spaces', 'num_digits', 
                                                'num_words', 'num_unique_words', 'avg_word_length',
                                                'num_punctuation', 'num_uppercase_letters', 'num_uppercase_words',
                                                'num_stop_words', 'num_sentences', 'num_hashtags',
                                                'num_capslock', 'num_Edit', 'num_not_ASCII', 'freq_punctuation', 'num_emoji'])

    return result_df

In [ ]:
train_df = analyze_comments(train_df, max_workers)

In [ ]:
# Чистка текста

demoji.download_codes()
p = inflect.engine()
dictionary = enchant.Dict("en_US")

# def correct_spelling(word, dictionary):
#     if dictionary.check(word):
#         return word
#     else:
#         suggestions = dictionary.suggest(word)
#         if len(suggestions) > 0:
#             return suggestions[0]
#         else:
#             return word

#     # Tokenize words
#     words = word_tokenize(text)
    
#     # Remove stop words
#     words = [w for w in words if not w in stop_words]
    
#     # Correct spelling
#     corrected_words = [correct_spelling(word, dictionary) for word in words]
    
#     corrected_words = [
#         p.number_to_words(word) if word.isdigit() and int(word) <= 100000 else word
#         for word in corrected_words
#     ]
    

def replace_emojis(text):
    
    return demoji.replace_with_desc(text)


def remove_symbols(text):
    
    text = re.sub(r'[^\w\s.,?!]', '', text)
    return text


def clean_text(text):
    
    text = text.replace('\n', ' ')
    cleaned_text, links_string, num_links = extract_links_and_clean_text(text)
    cleaned_text = demoji.replace_with_desc(cleaned_text)
    cleaned_text = cleaned_text.encode('ascii', 'ignore').decode()
    cleaned_text = cleaned_text.strip()
    return cleaned_text, links_string or None, num_links or None

def clean_chunk(df, column_name, index_slice):
    cleaned_chunk = []
    links_chunk = []
    num_links_chunk = []
    for i in index_slice:
        cleaned_comment, links_string, num_links = clean_text(df.loc[i, column_name])
        cleaned_chunk.append(cleaned_comment)
        links_chunk.append(links_string)
        num_links_chunk.append(num_links if num_links is not None else 0)
    return cleaned_chunk, links_chunk, num_links_chunk

def clean_comments(df, column_name, max_workers):
    cleaned_comments = []
    links_strings = []
    num_links = []

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        index_chunks = np.array_split(df.index, 10000)
        results = [executor.submit(clean_chunk, df, column_name, index_slice) for index_slice in index_chunks]
        for result in tqdm(concurrent.futures.as_completed(results), total=len(index_chunks)):
            cleaned_chunk, links_chunk, num_links_chunk = result.result()
            cleaned_comments.extend(cleaned_chunk)
            links_strings.extend(links_chunk)
            num_links.extend(num_links_chunk)

    cleaned_df = pd.DataFrame({
        'cleaned_' + column_name: cleaned_comments,
        'links_from_' + column_name: links_strings,
        'num_links_from_' + column_name: num_links,
        'index_col': df.index
    })
    # Sort by index
    cleaned_df = cleaned_df.sort_values(by='index_col')
    df = df.sort_index()
    cleaned_df = cleaned_df.set_index('index_col')
    df = df.join(cleaned_df)
    return df

def clean_html_entities(text):
    return html.unescape(text)

In [ ]:
train_df = clean_comments(train_df, 'comment', max_workers)
train_df = clean_comments(train_df, 'post_text', max_workers)

In [ ]:
train_df['cleaned_comment'] = train_df['cleaned_comment'].apply(clean_html_entities)
train_df['cleaned_post_text'] = train_df['cleaned_post_text'].apply(clean_html_entities)

In [ ]:
# Проверка на отсутствие пустых ячеек
print("Проверка в комментариях")
print("Всего строк:", train_df['cleaned_comment'].count())
print("Из них не пустых:", train_df['cleaned_comment'].astype(str).apply(lambda x: len(x.strip()) > 0).sum())
print("\nПроверка в постах")
print("Всего строк:", train_df['cleaned_post_text'].count())
print("Из них не пустых:", train_df['cleaned_post_text'].astype(str).apply(lambda x: len(x.strip()) > 0).sum())

In [ ]:
# Если пустые есть, заменяем их на оригинальные
def replace_empty_comment_with_original(df, cleaned_text, original_text):
    for i, row in tqdm(df.iterrows()):
        if len(str(row[cleaned_text]).strip()) < 1:
            df.loc[i, cleaned_text] = row[original_text]
    return df

train_df  = replace_empty_comment_with_original(train_df, 'cleaned_comment', 'comment')
train_df  = replace_empty_comment_with_original(train_df, 'cleaned_post_text', 'post_text')

In [ ]:
# Добавление дополнительных фич в виде распознанных в тексте объектов из модели en_core_web_sm
def add_entity_column(df, column_name, max_workers):
    nlp = spacy.load("en_core_web_sm")
    entity_types = ["PERSON", "FAC", "ORG", "GPE", "PRODUCT", "EVENT", "WORK_OF_ART", "LAW", "MONEY"]
    for entity_type in entity_types:
        new_column_name = f"entity_{entity_type}_{column_name}"
        df[new_column_name] = 0
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        for i, row in tqdm(df.iterrows(), total=len(df)):
            text = row[column_name]
            entity_count = process_row(text, nlp)
            for entity_type in entity_types:
                new_column_name = f"entity_{entity_type}_{column_name}"
                df.at[i, new_column_name] = entity_count[entity_type]
    return df

def process_row(text, nlp):
    entity_count = {"PERSON": 0, "FAC": 0, "ORG": 0,
                    "GPE": 0, "PRODUCT": 0, "EVENT": 0,
                    "WORK_OF_ART": 0, "LAW": 0, "MONEY": 0}
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ in entity_count:
            entity_count[ent.label_] += 1
    return entity_count

In [ ]:
train_df = add_entity_column(train_df, 'cleaned_post_text', max_workers=max_workers)
train_df = add_entity_column(train_df, 'cleaned_comment', max_workers=max_workers)

In [ ]:
train_df['main_cleaned_comment']

In [ ]:
# Проверка на наличие пустых строк
filtered_df = train_df[train_df['cleaned_comment'].apply(lambda x: len(x.strip()) < 1)]
print(filtered_df['cleaned_comment'])


In [ ]:
import plotly.express as px

# Распределение очищенных комментариев в зависимости от их длины
fig = px.histogram(train_df, x=train_df['cleaned_comment'].str.len(), nbins=50,
                   title='Distribution of Comment Lengths')
fig.show()


In [ ]:
# Выделение значимости предложения согласно TextBlob до 300 символов
from textblob import TextBlob

def extract_main_gist(comment):
    # Use TextBlob to extract keywords and noun phrases
    blob = TextBlob(comment)
    keywords = [word for word, tag in blob.tags if tag in ['NN', 'NNP', 'NNS', 'NNPS', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']]
    noun_phrases = [np for np in blob.noun_phrases]
    summary = ' '.join(keywords + noun_phrases)[:300] + '...'  # Combine and truncate to 300 characters
    return summary

# def correct_text(text):
#     blob = TextBlob(text)
#     return str(blob.correct())

def get_sentiment(comment):
    blob = TextBlob(comment)
    sentiment = blob.sentiment.polarity
    return sentiment

tqdm.pandas()

def process_comments(df, column, max_workers):
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        # Распараллеливание выполнения функции extract_main_gist
        main_comments = list(tqdm(executor.map(extract_main_gist, df[column]), total=len(df)))
        
        # Распараллеливание выполнения функции get_sentiment
        sentiments = list(tqdm(executor.map(get_sentiment, df[column]), total=len(df)))
    
    new_column_main = 'main_' + column
    new_column_sent = 'sentiment_' + column
    df[new_column_main] = main_comments
    df[new_column_sent] = sentiments
    return df

In [ ]:
train_df = process_comments(train_df, 'cleaned_comment', max_workers)
train_df = process_comments(train_df, 'cleaned_post_text', max_workers)

In [ ]:
# еще раз посмотрим на распределение
import plotly.express as px

fig = px.histogram(train_df, x=train_df['main_cleaned_comment'].str.len(), nbins=50,
                   title='Distribution of Comment Lengths')
fig.show()

In [ ]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

def chunk_text(text):
    max_length = 512
    chunks = []
    while len(text) > max_length:
        chunk = text[:max_length]
        chunks.append(chunk)
        text = text[max_length:]
    chunks.append(text)
    return chunks


def calculate_similarities(post_text, comment_text):
    # Initialize the HashingVectorizer
    vectorizer = HashingVectorizer(n_features=2 ** 10)

    max_similarities = [0.0] * 3

    post_vector = vectorizer.transform([post_text])

    def process_comment(comment_text):
        # Split the comment text into chunks of up to 512 tokens
        comment_chunks = chunk_text(comment_text)
        # Calculate the similarity score for each chunk
        chunk_scores = []
        for chunk in comment_chunks:
            comment_vector = vectorizer.transform([chunk])
            # Calculate the cosine similarity 
            cosine_sim = cosine_similarity(post_vector, comment_vector)[0][0]
            # Calculate the Jaccard similarity 
            comment_vector_array = comment_vector.toarray()[0]
            post_vector_array = post_vector.toarray()[0]
            intersection = np.logical_and(post_vector_array, comment_vector_array).sum()
            union = np.logical_or(post_vector_array, comment_vector_array).sum()
            jaccard_sim = intersection / union if union != 0 else 0
            # Calculate the Euclidean distance 
            euclidean_dist = euclidean_distances(post_vector, comment_vector)[0][0]

            chunk_scores.append((cosine_sim, jaccard_sim, euclidean_dist))

        comment_similarity = sorted(
            chunk_scores, key=lambda x: x[0], reverse=True
        )[:3]
        return [similarity[0] for similarity in comment_similarity]

    similarities = process_comment(comment_text)

    # Find the maximum similarities
    for i, similarity in enumerate(similarities):
        if similarity > max_similarities[i]:
            max_similarities[i] = similarity

    return max_similarities

def add_similarities(df):
    # Add new columns for the max similarity scores
    similarities = []
    for i, row in tqdm(df.iterrows(), total=df.shape[0]):
        max_similarities = calculate_similarities(row["cleaned_post_text"], row["cleaned_comment"])
        similarities.append(max_similarities)
    df[["max_cosine_sim", "max_jaccardi_sim", "max_euclidean_dist"]] = pd.DataFrame(similarities)
    return df



In [ ]:
train_df = add_similarities(train_df)

In [ ]:
# Функция для подсчета новых строк очищенных коротких комментариев - только значимые фичи
def count_main_features(text):
    num_chars_com = len(text) # число символов в строке
    num_punctuation = len([char for char in text if char in string.punctuation]) # число знаков препинания в строке
    num_capslock = len([word for word in text.split() if word.isupper()]) # число слов в верхнем регистре
    num_unique_words = len(set(text.split())) # число уникальных слов
    freq_punctuation = num_punctuation/num_chars_com # частота знаков препинания

    return pd.Series([num_chars_com, num_punctuation, num_capslock, num_unique_words, freq_punctuation])

train_df[['num_chars_com_main', 'num_punctuation_main', 'num_capslock_main', 'num_unique_words_main', 'freq_punctuation_main']] = train_df['main_cleaned_comment'].apply(count_main_features)


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

def normalize_columns(df):
    # Определение столбцов с числовыми данными, кроме столбцов id и score
    numeric_cols = [col for col in df.columns if col not in ['id', 'score'] and df[col].dtype in ['float64', 'int64']]
    
    for col in numeric_cols:
        scaler = StandardScaler()
        df[col] = scaler.fit_transform(df[[col]])

    return df

train_df = normalize_columns(train_df)

In [ ]:
# посчитаем относительную позицию интересующих показателей по группам id

train_df['relative_num_chars'] = train_df.groupby('id')['num_chars_com_main'].transform(lambda x: x.rank(method='dense') / x.shape[0])

train_df['relative_freq_punctuation'] = train_df.groupby('id')['freq_punctuation_main'].transform(lambda x: x.rank(method='dense') / x.shape[0])

train_df['relative_capslock'] = train_df.groupby('id')['num_capslock'].transform(lambda x: x.rank(method='dense') / x.shape[0])

train_df['relative_unique_words'] = train_df.groupby('id')['num_unique_words_main'].transform(lambda x: x.rank(method='dense') / x.shape[0])

train_df['relative_emoji'] = train_df.groupby('id')['num_emoji'].transform(lambda x: x.rank(method='dense') / x.shape[0])

train_df['relative_cosine_sim'] = train_df.groupby('id')['max_cosine_sim'].transform(lambda x: x.rank(method='dense') / x.shape[0])

train_df['relative_links_from_comment'] = train_df.groupby('id')['num_links_from_comment'].transform(lambda x: x.rank(method='dense') / x.shape[0])


In [ ]:
train_df.to_csv('test_df_norm.csv', index=False)

**Анализ данных**

In [ ]:
sns.set(style="darkgrid")

numeric_cols = [col for col in train_df.columns if col not in ['id', 'score'] and train_df[col].dtype in ['float64', 'int64']]
num_cols_per_row = 3
num_rows = int(np.ceil(len(numeric_cols) / num_cols_per_row))

fig, axes = plt.subplots(num_rows, num_cols_per_row, figsize=(15, 5*num_rows))
axes = axes.flatten()

for i, col in enumerate(numeric_cols):
    sns.violinplot(y=col, x="score", data=train_df, palette="Pastel1", ax=axes[i])
    axes[i].set_title(f"Violin plot for {col}")

for ax in axes[len(numeric_cols):]:
    ax.remove()

plt.tight_layout()
plt.show()

In [ ]:
import plotly.express as px
def get_top_abs_correlations(df, cor, top_n=60):
    # Получить только числовые столбцы
    numeric_cols = [col for col in df.columns if df[col].dtype in ['float64', 'int64']]
    
    # Вычислить корреляции и отбросить избыточные пары
    corr_df = df.corr(method='pearson').stack().reset_index()
    corr_df.columns = ['feature_1', 'feature_2', 'corr']
    corr_df = corr_df[corr_df['feature_1'] < corr_df['feature_2']]
    
    # Выбрать топ корреляций по значению
    corr_df = corr_df[(corr_df['corr'] >= cor) | (corr_df['corr'] <= -cor)]
    corr_df = corr_df.sort_values(by='corr', ascending=False).head(top_n)
    
    # Создать heatmap с выбранными парами и отображением значений корреляции
    fig = px.imshow(corr_df.pivot('feature_1', 'feature_2', 'corr'),
                     labels=dict(x='Feature 1', y='Feature 2', color='Correlation'))
    fig.update_layout(width=800, height=800, font=dict(size=8), 
                      title=f'Top Correlated Features (from: -{cor} to: -1 & from: {cor} to 1)')
    fig.show()


# Использование функции
get_top_abs_correlations(train_df, 0.7)

In [ ]:
# Определим лист наиболее интересных переменных

list_of_interes = ['id', 'post_text', 'comment', 'score', 'num_chars_com',
       'num_chars_post', 'num_punctuation', 'num_capslock', 'num_emoji', 'cleaned_comment',
       'links_from_comment', 'num_links_from_comment', 'main_cleaned_comment','cleaned_post_text',
       'main_cleaned_post_text', 'max_cosine_sim',
       'entity_PERSON_cleaned_comment', 'entity_FAC_cleaned_comment', 
       'entity_GPE_cleaned_comment', 'entity_PRODUCT_cleaned_comment',
       'entity_LAW_cleaned_comment', 'entity_MONEY_cleaned_comment',
       'num_chars_com_main', 'num_punctuation_main', 'num_capslock_main',
        'relative_num_chars', 'relative_freq_punctuation']

list_of_model = ['id', 'score', 'num_chars_com',
       'num_chars_post', 'num_punctuation', 'num_capslock', 'num_emoji', 'cleaned_comment',
        'num_links_from_comment', 'cleaned_post_text','max_cosine_sim',
       'entity_PERSON_cleaned_comment', 'entity_FAC_cleaned_comment', 
       'entity_GPE_cleaned_comment', 'entity_PRODUCT_cleaned_comment',
       'entity_LAW_cleaned_comment', 'entity_MONEY_cleaned_comment',
       'num_chars_com_main', 'num_punctuation_main', 'num_capslock_main',
        'relative_num_chars', 'relative_freq_punctuation']

list_for_delete_outliers = ['num_emoji', 'num_links_from_comment', 'max_cosine_sim', 'entity_PERSON_cleaned_comment', 'entity_FAC_cleaned_comment', 'entity_LAW_cleaned_comment']

In [ ]:
train_df = train_df[list_of_model]
train_df.to_csv('test_for_model.csv', index=False)

**Model**

In [ ]:
from sklearn.model_selection import GroupShuffleSplit
import random
# Сплит
class DataPreparation:
    
    def __init__(self, train_df):
        self.train_df = train_df
        self.object_columns = train_df.select_dtypes(include=['object']).columns.tolist()
        self.ttrain_data = self.train_df.drop(columns=self.object_columns)
        
    def prepare_data(self):
        groups = [self.ttrain_data for _, self.ttrain_data in self.ttrain_data.groupby('id')]
        import random
        random.shuffle(groups)

        # мешаем комментарии
        for i in range(len(groups)):
            groups[i] =  groups[i].sample(frac=1)

        self.ttrain_data = pd.concat(groups).reset_index(drop=True)
        gss = GroupShuffleSplit(test_size=.30, n_splits=1, random_state=7).split(self.ttrain_data, groups=self.ttrain_data['id'])
        X_train_inds, X_test_inds = next(gss)

        traindata = self.ttrain_data.iloc[X_train_inds]
        X_train = traindata.loc[:, ~traindata.columns.isin(['id','score'])]
        y_train = traindata.loc[:, traindata.columns.isin(['score'])]

        groups = traindata.groupby('id').size().to_frame('size')['size'].to_numpy()

        testdata = self.ttrain_data.iloc[X_test_inds]

        #We need to keep the id for later predictions
        X_test = testdata.loc[:, ~testdata.columns.isin(['score'])]
        y_test = testdata.loc[:, testdata.columns.isin(['score'])]

        return X_train, y_train, X_test, y_test, groups



In [ ]:
data_prep = DataPreparation(train_df)
X_train, y_train, X_test, y_test, groups = data_prep.prepare_data()

In [ ]:
class XGBRankerWrapper:
    def __init__(self, tree_method='hist', booster='gbtree', objective='rank:ndcg', random_state=42, 
                 learning_rate=0.01, colsample_bytree=1, max_depth=10, n_estimators=110, subsample=0.5):
        self.model = xgb.XGBRanker(
            tree_method=tree_method,
            booster=booster,
            objective=objective,
            random_state=random_state,
            learning_rate=learning_rate,
            colsample_bytree=colsample_bytree,
            max_depth=max_depth,
            n_estimators=n_estimators,
            subsample=subsample
        )

    def fit(self, X_train, y_train, groups, verbose=True):
        self.model.fit(X_train, y_train, group=groups, verbose=verbose)
    
    def predict(self, X_test):
        return self.model.predict(X_test.loc[:, ~X_test.columns.isin(['id'])])
        
def form(line, id_counter):
    rankings = line[1].rankings
    id_value = next(id_counter)
    return pd.DataFrame({'id': [id_value]*5, 'score': rankings})

def get_result(y):
    res = []
    id_counter = iter(y['id'])
    for line in y.iterrows():
        res.append(form(line, id_counter))
    result = pd.concat(res, ignore_index=True)
    return result

In [ ]:
model = XGBRankerWrapper()
data_prep = DataPreparation(train_df)
X_train, y_train, X_test, y_test, groups = data_prep.prepare_data()
model.fit(X_train, y_train, groups)
predictions = (X_test.groupby('id').apply(lambda x: model.predict(x)))
y = pd.DataFrame({'id':predictions.index, 'rankings':predictions.values})
for i in range(len(y['rankings'])):
    y['rankings'][i] = y['rankings'][i].argsort().argsort()
result = get_result(y)
result['true_score'] = y_test['score'].tolist()

**Metrics**

In [ ]:
import plotly.express as px

def get_score(result):
    dff = result.groupby(['id'])
    ids = result.id.unique()

    ndcg = []
    for i in range(len(ids)):
        a = ndcg_score(np.asarray([dff.get_group(ids[i])['true_score']]), np.asarray([dff.get_group(ids[i])['score']]))
        ndcg.append(a)

    fig = px.histogram(ndcg, nbins=50, opacity=0.66)
    fig.update_layout(title={'text': 'Распределение ndcg-скоров', 'x': 0.5, 'y': 0.8}, 
                      xaxis_title='ndcg-скор', yaxis_title='Количество', bargap=0.1)
    fig.show()

    scor = sum(ndcg) / len(ndcg)
    return scor

get_score(result)

In [ ]:
from tqdm import tqdm

class XGBRankerGridSearch:
    def __init__(self, params, params_grid):
        self.params = params
        self.params_grid = params_grid
        self.learning_rate = params['learning_rate']
        self.grow_policy = params['grow_policy']

    def fit(self, X_train, y_train, groups, X_test, y_test):
        model_pars = []
        ndcg_score = []
        for i in tqdm(range(len(self.params_grid['learning_rate']))):
            self.params.update({'learning_rate': self.params_grid['learning_rate'][i]})
            for k in tqdm(range(len(self.params_grid['grow_policy']))):
                self.params.update({'grow_policy': self.params_grid['grow_policy'][k]})
                print([self.params['learning_rate'], self.params['grow_policy']])

                #model_pars.append([params_grid['learning_rate'][i], params_grid['grow_policy'][k]])
                model_pars.append([self.params['learning_rate'], self.params['grow_policy']])
                model = xgb.XGBRanker(tree_method = self.params['tree_method'],
                                      objective = self.params['objective'],
                                      colsample_bytree = self.params['colsample_bytree'],
                                      subsample = self.params['subsample'],
                                      n_estimators = self.params['n_estimators'],
                                      random_state = self.params['random_state'], 
                                      learning_rate = self.params['learning_rate'],
                                      booster = self.params['booster'],
                                      grow_policy = self.params['grow_policy'],
                                      max_depth = self.params['max_depth']
                                     )
                
                model.fit(X_train, y_train, group=groups, verbose=True)
                
                def predict(model, df):
                    return model.predict(df.loc[:, ~df.columns.isin(['id'])])
                
                predictions = (X_test.groupby('id').apply(lambda x: predict(model, x)))

                y = pd.DataFrame({'id':predictions.index, 'rankings':predictions.values})
                for i in range(len(y['rankings'])):
                  y['rankings'][i] = y['rankings'][i].argsort().argsort()
                
                def form(line, id_counter):
                    rankings = line[1].rankings
                    id_value = next(id_counter)
                    return pd.DataFrame({'id': [id_value]*5, 'score': rankings})

                def get_result(y):
                    res = []
                    id_counter = iter(y['id'])
                    for line in y.iterrows():
                        res.append(form(line, id_counter))
                    result = pd.concat(res, ignore_index=True)
                    return result

                result = get_result(y)
                result['true_score'] = y_test['score'].tolist()
                
                ndcg_score.append(get_score(result))
                  
        print(model_pars)
        print(ndcg_score)
        max_score = max(ndcg_score)
        print(max_score)
        max_ind = ndcg_score.index(max_score)
        print(max_ind)
        self.params.update({'learning_rate': model_pars[max_ind][0]})
        self.params.update({'grow_policy': model_pars[max_ind][1]})


In [ ]:
params = {
    'tree_method': 'gpu_hist',
    'objective': 'rank:ndcg',
    'colsample_bytree': 0.8,
    'subsample': 0.8,
    'n_estimators': 100,
    'random_state': 42,
    'learning_rate': 0.1,
    'booster': 'gbtree',
    'grow_policy': 'lossguide',
    'max_depth': 6
}

params_grid = {
    'learning_rate': [0.1, 0.05, 0.01],
    'grow_policy': ['depthwise', 'lossguide']
}

search = XGBRankerGridSearch(params, params_grid)
search.fit(X_train, y_train, groups, X_test, y_test)

In [ ]:
# обновление параметров
params.update({'learning rate': model_pars[0]})
params.update({'grow_policy': model_pars[1]})

In [ ]:
from matplotlib import pyplot as plt
from xgboost import plot_importance

def my_plot_importance(booster, figsize): 
    fig, ax = plt.subplots(1,1,figsize=figsize)
    return plot_importance(booster=booster, ax=ax, importance_type='cover')

my_plot_importance(model, (30,15))

In [ ]:
#выбор оптимальных фич
terf = pd.DataFrame()
terf['id'] = train_data['id']
terf['score'] = train_data['score']
terf['num_chars_com'] = train_data['num_chars_com']
terf['num_punctuation_main'] = train_data['num_punctuation_main']
terf['relative_freq_punctuation'] = train_data['relative_freq_punctuation']


In [ ]:
# Модель с новыми параметрами
model = XGBRankerWrapper()
data_prep = DataPreparation(terf)
X_train, y_train, X_test, y_test, groups = data_prep.prepare_data()
model.fit(X_train, y_train, groups)
predictions = (X_test.groupby('id').apply(lambda x: model.predict(x)))
y = pd.DataFrame({'id':predictions.index, 'rankings':predictions.values})
for i in range(len(y['rankings'])):
    y['rankings'][i] = y['rankings'][i].argsort().argsort()
result = get_result(y)
result['true_score'] = y_test['score'].tolist()
get_score(result)

In [ ]:
test = pd.DataFrame()
test['id'] = train_df['id']
test['score'] = train_df['score']
test['num_chars_com'] = train_df['num_chars_com']
test['num_punctuation_main'] = train_df['num_punctuation_main']
test['relative_freq_punctuation'] = train_df['relative_freq_punctuation']

In [ ]:
y_train

In [ ]:
X_test = test[['id', 'num_chars_com', 'num_punctuation_main', 'relative_freq_punctuation']]
y_test = test[['score']]
predictions = (X_test.groupby('id').apply(lambda x: model.predict(x)))
y = pd.DataFrame({'id':predictions.index, 'rankings':predictions.values})
for i in range(len(y['rankings'])):
    y['rankings'][i] = y['rankings'][i].argsort().argsort()
result = get_result(y)
result['true_score'] = y_test['score'].tolist()
result

In [ ]:
import copy
a = np.empty((70020))
a[:] = np.nan
A = copy.deepcopy(test_df.drop(columns=['score']))
A['score'] = a
B = copy.deepcopy(result['score'])
A['score'] = B.tolist()
ids = A.id.unique()
grs = A.groupby(['id'])
grs.get_group(ids[0])

In [ ]:
df_list = []
for i in range(len(ids)):
  line = dict()
  com_line = []
  comments = [i for i in grs.get_group(ids[i])["comment"]]
  scores = [i for i in grs.get_group(ids[i])["score"]]
  text = grs.get_group(ids[i])["text"].iloc[0]
  for j in range(5):
    comm = dict()
    comm["text"] = comments[j]
    comm["score"] = scores[j]
    com_line.append(comm)
  line["text"] = text
  line["comments"] = com_line
  df_list.append(line)

with open('ranking_test.jsonl', 'w') as f:
  for j in df_list:
    f.write(str(json.dumps(j)))
    f.write('\n')
    
f.close()